In [ ]:
%%HTML
<script>
  function code_toggle() {
    if (code_shown){
      $('div.input').hide('200');
      $('#toggleButton').val('Show Code')
    } else {
      $('div.input').show('200');
      $('#toggleButton').val('Hide Code')
    }
    code_shown = !code_shown
  }

  $( document ).ready(function(){
    code_shown=true;
    code_toggle()
  });
</script>
<form action="javascript:code_toggle()"><input type="submit" id="toggleButton" value="Show Code"></form>

In [ ]:
%%html  
<style>.cm-s-ipython .CodeMirror-matchingbracket { color: black !important;}</style><!-- Bob J: no green highlights -->
<style>table.dataframe th { vertical-align: top; }</style><!-- Move MultiIndex headers to top of block -->
<style>table.dataframe td, table.dataframe th { border-style: solid; border-width: thin; }</style><!-- thin border around tables -->

# CSC Exercises Summary Data
All times are UTC

In [ ]:
%%javascript  # Prefer to display output instead of scrolling, so it can print
IPython.OutputArea.prototype._should_scroll = function(lines) {
    return false;
}

In [ ]:
# Standard definitions and options
import pandas as pd
from pandas import DataFrame
import os
import numpy as np
from datetime import datetime
%matplotlib inline
import matplotlib.pyplot as plt
plt.rcParams['figure.figsize'] = (15.0, 10.0)
# widen the page to match the window
from IPython.display import display, HTML
display(HTML("<style>.container { width:100% !important; }</style>"))
# full width display of tables
pd.options.display.max_columns = 50
# want to display all rows
pd.options.display.max_rows = None

import warnings
warnings.simplefilter(action='ignore', category=FutureWarning)

import json
# read log in JSON form, as processed by i.e. transfer.php
with open('log.json', 'r') as f:
    samples_dict = json.load(f)
# drop items before School starts? What if somebody went early?
# df = df[df.DATE > "2022-09-02 15:30:00"]
# drop my entries?
# df = df[df.user != 'jake']
# df = df[df.user != 'bojacobs']
# samples_dict

In [ ]:
exerciseName = {
    "TestIntroJava":       " 1 Testing 1 Intro Java",
    "TestIntroCpp":        " 3 Testing 3 Intro C++",
    "TestIntroPy":         " 2 Testing 2 Intro Python",
    "TestSumPrimesJava":   " 4 Testing 4 TestSumPrimes Java",
    "TestSumPrimesPy":     " 5 Testing 5 TestSumPrimes Python",
    "TestSumPrimesCpp":    " 6 Testing 6 TestSumPrimes C++",
    "ProfileStatsJava":    " 7 Performance 1 Java",
    "ProfileStatsCpp":     " 8 Performance 2 C++",
    "MemIntroCpp":         " 9 Memory 1 C++ Memory Checks",
    "GitBasics1":          "10 Git 1 GitBasics1",
    "GitBasics2":          "11 Git 2 GitBasics2",
    "GitBisect":           "12 Git 3 GitBisect",
    "GitHeadBanging":      "13 Git 4 GitHeadBanging"
}


In [ ]:
df = DataFrame()
for sample in samples_dict : 
    df = df.append(sample, ignore_index=True)
df = df.replace(np.nan, '', regex=True)

# Add column
def lookUpCanFail(array, item, failValue) :
    try :
        return array[item]
    except:
        return failValue

df['Exercise'] = df.apply(lambda row: \
                    lookUpCanFail(exerciseName, row.item2, ""), axis = 1)

#df.head(8)

### Team Names

In [ ]:
# extract team names, then drop them
teams = (df[df["item1"]=="team"])[['user', 'item1', 'item2', 'item3']]
display(teams)
df = df[df["item1"]!="team"]

### How many times was each exercise started?

In [ ]:
# how many total times for each Exercise
temp = df[["Exercise", "user"]].copy()
t1 = temp[temp.Exercise.notnull()]
t1 = t1[t1.Exercise != ""]
group = t1.groupby(["Exercise"]).count()
count = group['user']
display(group)
fig, ax = plt.subplots(figsize =(16, 9))
ax.barh(group.index, count)
ax.invert_yaxis()
ax.set_title('Number Attempting Each Exercise');

### Machine usage information

In [ ]:
# show host names so you can e.g. see if somebody working locally
temp = df[['host', 'user']].copy()
#display(temp.groupby(["host", "user"]).count())
temp = df[['host']].copy()
display(temp.groupby(["host"]).count())

In [ ]:
temp = df[['IP']].copy()
#display(temp.groupby("IP").count())

In [ ]:
temp = df[['user', 'host']].sort_values('user').copy()
temp.groupby(["user", "host"]).count()

In [ ]:
# progress by each specific user
temp = df[['user', 'Exercise']].sort_values('user').copy()
#display(temp.groupby(["user", "Exercise"]).count())

In [ ]:
# look at one user
#display(df[df.user=="jake"])

### Time users started exercises, after the time needed for checkout

In [ ]:
# first 'setup' (startup) time - this is the start of working on the exercises
t1 = df[df.item1 == 'setup']
t1.drop_duplicates(['user'], keep='first')[["DATE", "IP", "host", "user"]]

In [ ]:
# plot the 'setup' (startup) time (when the student got to 'source setup', i.e. after checkout)
t1 = df[df.item1 == 'setup']

# restrict to exercise period 
# t1 = t1[t1.DATE > "2019-09-17 04:00:00"]

# Not dropping duplicates by user, should it? Above keeps just first. Setup when opening another xterm?

times = pd.to_datetime(t1["DATE"]) # already in order by time as log is written
count = np.arange(0, len(times))
plt.plot_date(times, count)
plt.gcf().autofmt_xdate()
plt.title("Time Running Setup")
plt.show()


### Most recent exercise by user

In [ ]:
# most recent 'Exercise' by user
t1 = df[df.Exercise.notnull()]
t2 = t1.drop_duplicates(['user'], keep='last').sort_values(['user'])
t2[['DATE','user','Exercise']]

Last 20 activities

In [ ]:
# activity near the end 
t2 = t1.drop_duplicates(['user'], keep='last').sort_values(['DATE'])
t2[['DATE','user','Exercise']].tail(20)

### How many times is each exercise the most recent (i.e. current) for people?

In [ ]:
# count the number of people with each exercise as most recent
temp = df[['user', 'Exercise']].sort_values('user').copy()
t1 = temp[temp.Exercise.notnull()]
t1 = t1[t1.Exercise != '']
t2 = t1.drop_duplicates(['user'], keep='last')
t2.sort_values("Exercise")
t2.groupby("Exercise").count()

In [ ]:
# t1.tail(5)  # t1 is exercise sequence

### When did users start their most recent exercise?

In [ ]:
# plot time users started their last exercise
t3 = df[df.item1.notnull()]
t3 = t3[t3.item1 != '']
t3 = t3.drop_duplicates(['user'], keep='last')
t3.sort_values("DATE")
display(t3.head())

#t3 = t3[t3.DATE < "2019-09-18 17:30:00"]
#t3 = t3[t3.DATE > "2019-09-18 15:30:00"]

times = pd.to_datetime(t3["DATE"])
count = np.arange(0, len(times))
plt.plot_date(times, count)
plt.gcf().autofmt_xdate()
plt.title("Time Starting Most Recent Exercise")
plt.show()

